In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [13]:
from tensor2tensor.utils import beam_search

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate,
                beam_width = 5):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(embeddings, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(vocab_size)
        
        with tf.variable_scope('decode'):
            decoder_cells = attention(encoder_out, self.X_seq_len)
            states = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)

            training_helper = tf.contrib.seq2seq.TrainingHelper(
                    inputs = tf.nn.embedding_lookup(embeddings, decoder_input),
                    sequence_length = self.Y_seq_len,
                    time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = training_helper,
                    initial_state = states,
                    output_layer = dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = training_decoder,
                    impute_finished = True,
                    maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
        
        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_out, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            decoder_cell = attention(encoder_out_tiled, X_seq_len_tiled, reuse=True)
            states = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(
                    cell_state = encoder_state_tiled)
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = states,
                beam_width = beam_width,
                output_layer = dense,
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.fast_result = predicting_decoder_output.predicted_ids[:, :, 0]
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [15]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, learning_rate)
sess.run(tf.global_variables_initializer())

In [16]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [17]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[ 2621,  1264,  1264,  1264,  1264, 16636, 16636, 16636, 16636,
         14378, 14378, 14378, 14378, 19760, 19760, 19760, 21908, 21908,
         21908, 21908, 21908, 13366, 13366, 13366, 13366,   448,   448,
           448, 22883, 22883, 22883, 22883, 29429, 29429, 29429, 24660,
         24660, 24660, 24660, 31500, 31500,  8616,  8881,  8881,  3118,
          3118,  3118,  2160,  2160,  2160, 17274, 17274, 17274, 17274,
         28792, 28792, 28792, 28792, 28792, 28792, 28713, 28713, 28713,
         28713, 28713, 28713,  4153,  4153,  4153,  4153, 23355, 23355],
        [ 9986,  3404,  3111,  3111,  3111,  8273,  8273,  8273, 21183,
         21183, 21183, 28782, 28782,  7649,  7649,  7649,  7649, 23993,
         23993,  1843,  1843,  5667,  5667,  5667,  5667,  5667,  5667,
          5667, 13397, 13397,  5667, 13397, 13397, 13397, 13397, 13397,
         21492, 21492, 21492,  2193,  9981,  9981,  9981,  9981,  9981,
          9981,  9981, 26799, 27094, 27094,  5556,  5556,  5556

In [18]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 5.263012, training avg acc 0.226258
epoch 1, testing avg loss 3.926299, testing avg acc 0.355242


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 3.384498, training avg acc 0.420094
epoch 2, testing avg loss 3.069542, testing avg acc 0.458404


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 18, training avg loss 0.842631, training avg acc 0.784520
epoch 18, testing avg loss 3.598517, testing avg acc 0.492979


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 19, training avg loss 0.801168, training avg acc 0.792683
epoch 19, testing avg loss 3.687797, testing avg acc 0.490220


minibatch loop: 100%|██████████| 40/40 [00:07<00:00,  5.16it/s, accuracy=0.495, cost=3.78]

epoch 20, training avg loss 0.762615, training avg acc 0.800212
epoch 20, testing avg loss 3.755179, testing avg acc 0.488755


In [19]:
from tensor2tensor.utils import bleu_hook

In [20]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 40/40 [00:26<00:00,  1.51it/s]


In [21]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [22]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.17535137

In [23]:
rights[0], results[0]

([943,
  2610,
  409,
  925,
  9,
  346,
  289,
  3373,
  264,
  648,
  30773,
  391,
  9514,
  10050,
  18603,
  445,
  289,
  1451,
  325,
  3299,
  312,
  289,
  2196,
  317,
  1656,
  28736,
  12770,
  1561,
  336,
  346,
  321,
  17566,
  11387],
 [648,
  671,
  409,
  3421,
  610,
  346,
  289,
  4084,
  264,
  648,
  30773,
  3437,
  300,
  9514,
  10050,
  18603,
  376,
  289,
  1451,
  325,
  3299,
  312,
  289,
  2196,
  317,
  1656,
  28736,
  26,
  1561,
  336,
  346,
  321,
  6341,
  11387])